# 📥 Ingestão e Mapeamento das Tabelas

## ✅ Objetivo do Notebook

Este notebook executa as seguintes tarefas principais:

---

### 1. 🔍 Comparação entre Datasets de Mesmo Tipo

Verifica se estruturas de **tabelas semelhantes podem ser empilhadas**, garantindo a consistência do schema entre tabelas fragmentadas.

---

### 2. 📝 Criação de Arquivo `.json` com a Descrição das Tabelas

Gera automaticamente um **arquivo com metadados estruturais** de cada tabela, incluindo nomes de colunas, tipos de dados, totais e presença de chave primária.

---

### 3. 📖 Geração de Dicionário de Variáveis (em Português)

Cria um **mapeamento interpretável das variáveis**, facilitando a compreensão dos nomes técnicos e tornando a análise mais acessível.


### Criação da SparkSession

In [1]:
# Criação da SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploSparkSession") \
    .getOrCreate()

### Importações

In [2]:
from pyspark.sql.functions import col, when, count, isnan, isnull, countDistinct
import json
import pandas as pd
import os
import glob

In [3]:
diretorio = r"C:\Users\fred\meu_projeto_etl"

### 🔍Comparação entre datasets do mesmo tipo

In [12]:
## Carregamento das tabelas necessárias para validação de empilhamento
caminho = r"C:\Users\fred\meu_projeto_etl\data\raw\test"
test_base = spark.read.parquet(f"{caminho}\\test_base.parquet")
test_applprev_1_0 = spark.read.parquet(f"{caminho}\\test_applprev_1_0.parquet")
test_applprev_1_1 = spark.read.parquet(f"{caminho}\\test_applprev_1_1.parquet")
test_credit_bureau_a_1_0 = spark.read.parquet(f"{caminho}\\test_credit_bureau_a_1_0.parquet")
test_credit_bureau_a_1_1 = spark.read.parquet(f"{caminho}\\test_credit_bureau_a_1_1.parquet")
test_credit_bureau_a_2_0 = spark.read.parquet(f"{caminho}\\test_credit_bureau_a_2_0.parquet")
test_credit_bureau_b_1 = spark.read.parquet(f"{caminho}\\test_credit_bureau_b_1.parquet")
test_credit_bureau_b_2 = spark.read.parquet(f"{caminho}\\test_credit_bureau_b_2.parquet")
test_debitcard_1 = spark.read.parquet(f"{caminho}\\test_debitcard_1.parquet")
test_deposit_1 = spark.read.parquet(f"{caminho}\\test_deposit_1.parquet")
test_other_1 = spark.read.parquet(f"{caminho}\\test_other_1.parquet")
test_person_1 = spark.read.parquet(f"{caminho}\\test_person_1.parquet")
test_person_2 = spark.read.parquet(f"{caminho}\\test_person_2.parquet")
test_static_0_0 = spark.read.parquet(f"{caminho}\\test_static_0_0.parquet")
test_static_0_1 = spark.read.parquet(f"{caminho}\\test_static_0_1.parquet")
test_stati_cb_0 = spark.read.parquet(f"{caminho}\\test_static_cb_0.parquet")
test_tax_registry_a_1 = spark.read.parquet(f"{caminho}\\test_tax_registry_a_1.parquet")
test_tax_registry_b_1 = spark.read.parquet(f"{caminho}\\test_tax_registry_b_1.parquet")
test_tax_registry_c_1 = spark.read.parquet(f"{caminho}\\test_tax_registry_c_1.parquet")
test_credit_bureau_a_2_1 = spark.read.parquet(f"{caminho}\\test_credit_bureau_a_2_1.parquet")
test_credit_bureau_a_2_5 = spark.read.parquet(f"{caminho}\\test_credit_bureau_a_2_5.parquet")
test_credit_bureau_a_2_10= spark.read.parquet(f"{caminho}\\test_credit_bureau_a_2_10.parquet")

In [10]:
## Função para avaliar tabelas com colunas iguais
def comparar_tabelas(tabela1, tabela2):
    colunas1 = set(tabela1)
    colunas2 = set(tabela2)
    
    if colunas1 == colunas2:
        print("✅ Colunas iguais (ignora ordem).")
    else:
        print("✅ Colunas diferentes (ignora ordem).")

In [22]:
comparar_tabelas(test_applprev_1_0.columns, test_applprev_1_1.columns)

✅ Colunas iguais (ignora ordem).


In [7]:
comparar_tabelas(train_applprev_1_0.columns, train_applprev_1_1.columns)

✅ Colunas iguais (ignora ordem).


In [8]:
comparar_tabelas(train_credit_bureau_a_1_0.columns, train_credit_bureau_a_1_1.columns)

✅ Colunas iguais (ignora ordem).


In [11]:
comparar_tabelas(train_credit_bureau_a_1_0.columns, train_credit_bureau_a_2_0.columns)

✅ Colunas diferentes (ignora ordem).


In [12]:
comparar_tabelas(train_static_0_0.columns, train_static_0_1.columns)

✅ Colunas iguais (ignora ordem).


In [13]:
comparar_tabelas(train_tax_registry_a_1.columns, train_tax_registry_b_1.columns)

✅ Colunas diferentes (ignora ordem).


In [14]:
comparar_tabelas(train_credit_bureau_a_2_0.columns, train_credit_bureau_a_2_10.columns)

✅ Colunas iguais (ignora ordem).


## 📊 Resultado das Comparações
### 🧩 Estrutura dos Nomes dos Datasets

A nomenclatura dos arquivos segue um padrão que permite identificar suas subdivisões em grupos, com a seguinte estrutura:

(treino_ou_teste)_(fonte_de_dados)_(grupo_de_variáveis)(subgrupo)


treino_ou_teste: Indica se o dataset pertence ao conjunto de treino (train) ou teste (test).

fonte_de_dados: Define a origem da informação (por exemplo: credit_bureau, applprev, tax_registry, static).

grupo_de_variáveis: Agrupa variáveis com características semelhantes ou função analítica similar.

subgrupo (opcional): Representa uma subdivisão ainda mais específica, quando aplicável.

### 🧱 Possibilidade de Empilhamento

Os datasets que compartilham o mesmo nome até a parte de <grupo_de_variáveis> podem ser empilhados (concatenados) de forma segura, pois representam a mesma estrutura sem conflito semântico. Essa estratégia é útil para consolidação e análise integrada entre diferentes clientes ou aplicações.

## 📝 Criação de arquivo .json com a descrição das tabelas

In [5]:
# Caminho base dos arquivos
caminho = r"C:\Users\fred\meu_projeto_etl\data\raw\test"
caminho_base = caminho

# Lista dos grupos esperados
grupos = [
    "test_applprev_1",
    "test_applprev_2",
    "test_credit_bureau_a_1",
    "test_credit_bureau_a_2",
    "test_credit_bureau_b_1",
    "test_credit_bureau_b_2",
    "test_static_0",
    "test_base",
    "test_debitcard_1",
    "test_deposit_1",
    "test_other_1",
    "test_person_1",
    "test_person_2",
    "test_static_cb_0",
    "test_tax_registry_a_1",
    "test_tax_registry_b_1",
    "test_tax_registry_c_1"
]


### Empilhamento de tabelas

In [17]:
import os, glob, re
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col

def _dtype_map(df):
    return {f.name: f.dataType.simpleString() for f in df.schema.fields}

def _print_coluna_problemática(e_msg, dfA, dfB):
    colsA, colsB = dfA.columns, dfB.columns
    tiposA, tiposB = _dtype_map(dfA), _dtype_map(dfB)

    m = re.search(r"(\d+)(?:st|nd|rd|th) column", e_msg)
    if m:
        idx = int(m.group(1)) - 1
        nomeA = colsA[idx] if idx < len(colsA) else None
        nomeB = colsB[idx] if idx < len(colsB) else None
        if nomeA == nomeB:
            return [nomeA]

    comuns = sorted(set(colsA).intersection(colsB))
    diverg = [c for c in comuns if tiposA.get(c) != tiposB.get(c)]
    return diverg

def carregar_e_empilhar(grupo):
    padrao = os.path.join(caminho_base, f"{grupo}_*.parquet")
    arquivos = sorted(glob.glob(padrao))
    arquivo_unico = os.path.join(caminho_base, f"{grupo}.parquet")
    if os.path.exists(arquivo_unico):
        arquivos.append(arquivo_unico)
    if not arquivos:
        print(f"🚫 Nenhum arquivo encontrado para {grupo}")
        return None

    colunas_padrao = None
    dfs_validos, paths_validos = [], []

    for caminho_arquivo in arquivos:
        df = spark.read.parquet(caminho_arquivo)
        if colunas_padrao is None:
            colunas_padrao = df.columns
            dfs_validos.append(df.select(colunas_padrao))
            paths_validos.append(caminho_arquivo)
            continue

        set_padrao, set_atual = set(colunas_padrao), set(df.columns)
        if set_padrao != set_atual:
            faltando = sorted(set_padrao - set_atual)
            extras = sorted(set_atual - set_padrao)
            print(f"🚫 Colunas diferentes em: {caminho_arquivo}")
            if faltando:
                print(f"   FALTANDO: {faltando}")
            if extras:
                print(f"   EXTRAS  : {extras}")
            continue

        df = df.select(colunas_padrao)
        dfs_validos.append(df)
        paths_validos.append(caminho_arquivo)

    if not dfs_validos:
        print(f"⚠️ Nenhum DataFrame válido para {grupo}")
        return None

    df_final = dfs_validos[0]
    for i in range(1, len(dfs_validos)):
        df_i = dfs_validos[i]
        try:
            df_final = df_final.unionByName(df_i)
        except AnalysisException as e:
            print("\n⛔ Tipos incompatíveis detectados durante o union.")
            colunas_problema = _print_coluna_problemática(str(e), df_final, df_i)
            print(f"🔄 Convertendo colunas para string: {colunas_problema}")

            for col_name in colunas_problema:
                df_final = df_final.withColumn(col_name, col(col_name).cast("string"))
                df_i = df_i.withColumn(col_name, col(col_name).cast("string"))

            try:
                df_final = df_final.unionByName(df_i)
                print(f"✅ Union refeito com cast para string nas colunas: {colunas_problema}")
            except Exception as e2:
                print(f"❌ Falha mesmo após cast: {e2}")
                return None

    print(f"✅ {grupo} carregado com {len(dfs_validos)} arquivo(s).")
    return df_final


In [18]:
# Dicionário com os DataFrames carregados com sucesso
variaveis_criadas = {}

# Carregar todos os grupos
for grupo in grupos:
    df = carregar_e_empilhar(grupo)
    if df is not None:
        variaveis_criadas[grupo] = df

# Exibir nomes dos DataFrames criados
print("\n📦 Variáveis carregadas:")
for nome in variaveis_criadas:
    print(f" - {nome}")


⛔ Tipos incompatíveis detectados durante o union.
🔄 Convertendo colunas para string: ['isdebitcard_527L']
✅ Union refeito com cast para string nas colunas: ['isdebitcard_527L']

⛔ Tipos incompatíveis detectados durante o union.
🔄 Convertendo colunas para string: ['isdebitcard_527L']
✅ Union refeito com cast para string nas colunas: ['isdebitcard_527L']
✅ test_applprev_1 carregado com 3 arquivo(s).
✅ test_applprev_2 carregado com 1 arquivo(s).
✅ test_credit_bureau_a_1 carregado com 5 arquivo(s).
✅ test_credit_bureau_a_2 carregado com 12 arquivo(s).
✅ test_credit_bureau_b_1 carregado com 1 arquivo(s).
✅ test_credit_bureau_b_2 carregado com 1 arquivo(s).

⛔ Tipos incompatíveis detectados durante o union.
🔄 Convertendo colunas para string: ['isbidproductrequest_292L']
✅ Union refeito com cast para string nas colunas: ['isbidproductrequest_292L']
✅ test_static_0 carregado com 3 arquivo(s).
✅ test_base carregado com 1 arquivo(s).
✅ test_debitcard_1 carregado com 1 arquivo(s).
✅ test_deposit

In [21]:
test_applprev_1_0.count()

10

In [19]:
## Função para arquivo de descrição das tabelas
def analise_basica_spark_dict(df, nome_tabela, chave_primaria=None):
    schema = [(f.name, f.dataType.simpleString()) for f in df.schema.fields]
    n_regis = df.count()
    n_col = len(df.columns)
    print(f'Tabela: {nome_tabela}')
    print(f'Número de registros: {n_regis}')
    print(f'Número de colunas: {n_col}')
    describe_df = df.describe()
    print(describe_df)
    describe_data = {row["summary"]: {col: row[col] for col in row.asDict() if col != "summary"} for row in describe_df.collect()}

    nulls_expr = [
        ((count(when(col(c).isNull(), c)) / n_regis) * 100).alias(c)
        for c in df.columns
    ]
    nulls_df = df.select(nulls_expr)
    nulls_dict = nulls_df.collect()[0].asDict()
    nulls_filtrados = {k: round(v, 2) for k, v in nulls_dict.items() if v > 0}

    pk_status = "não verificada"
    if chave_primaria:
        if chave_primaria not in df.columns:
            pk_status = "coluna inexistente"
        else:
            distinct_count = df.select(chave_primaria).distinct().count()
            pk_status = "válida" if distinct_count == n_regis else "inválida"

    return {
        "tabela": nome_tabela,
        "total_registros": n_regis,
        "total_colunas": n_col,
        "schema": schema,
        "describe": describe_data,
        "colunas_com_nulos": nulls_filtrados,
        "chave_primaria": {
            "coluna": chave_primaria,
            "status": pk_status
        }
    }


In [20]:
# Acumulador de resultados
resultados = {}

# Alimentar resultados a cada execução
for nome, df in variaveis_criadas.items():
    resultado = analise_basica_spark_dict(df, nome_tabela=nome, chave_primaria="case_id")  
    resultados[nome] = resultado

Tabela: test_applprev_1
Número de registros: 30
Número de colunas: 41
DataFrame[summary: string, case_id: string, actualdpd_943P: string, annuity_853A: string, approvaldate_319D: string, byoccupationinc_3656910L: string, cancelreason_3545846M: string, childnum_21L: string, creationdate_885D: string, credacc_actualbalance_314A: string, credacc_credlmt_575A: string, credacc_maxhisbal_375A: string, credacc_minhisbal_90A: string, credacc_status_367L: string, credacc_transactions_402L: string, credamount_590A: string, credtype_587L: string, currdebt_94A: string, dateactivated_425D: string, district_544M: string, downpmt_134A: string, dtlastpmt_581D: string, dtlastpmtallstes_3545839D: string, education_1138M: string, employedfrom_700D: string, familystate_726L: string, firstnonzeroinstldate_307D: string, inittransactioncode_279L: string, isdebitcard_527L: string, mainoccupationinc_437A: string, maxdpdtolerance_577P: string, num_group1: string, outstandingdebt_522A: string, pmtnum_8L: string,

Tabela: test_base
Número de registros: 10
Número de colunas: 4
DataFrame[summary: string, case_id: string, date_decision: string, MONTH: string, WEEK_NUM: string]
Tabela: test_debitcard_1
Número de registros: 10
Número de colunas: 6
DataFrame[summary: string, case_id: string, last180dayaveragebalance_704A: string, last180dayturnover_1134A: string, last30dayturnover_651A: string, num_group1: string, openingdate_857D: string]
Tabela: test_deposit_1
Número de registros: 10
Número de colunas: 5
DataFrame[summary: string, case_id: string, amount_416A: string, contractenddate_991D: string, num_group1: string, openingdate_313D: string]
Tabela: test_other_1
Número de registros: 10
Número de colunas: 7
DataFrame[summary: string, case_id: string, amtdebitincoming_4809443A: string, amtdebitoutgoing_4809440A: string, amtdepositbalance_4809441A: string, amtdepositincoming_4809444A: string, amtdepositoutgoing_4809442A: string, num_group1: string]
Tabela: test_person_1
Número de registros: 10
Número 

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [22]:
# Salvar arquivo json com informações da descrição das tabelas
with open(fr"{diretorio}\\docs\analise_tabelas_test.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

In [24]:
# Carrega o JSON em um dicionário Python
with open(fr"{diretorio}\\docs\analise_tabelas_test.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

# Transformar o dicionário em uma lista de tabelas
dados_lista = list(dados.values())

# Criar DataFrame com o resumo das tabelas
df_resumo = pd.DataFrame(dados_lista)

from IPython.display import display

display(df_resumo.head(20))


,tabela,total_registros,total_colunas,schema,describe,colunas_com_nulos,chave_primaria
0,test_applprev_1,30,41,"[[case_id, bigint], [actualdpd_943P, double], ...","{'count': {'case_id': '30', 'actualdpd_943P': ...","{'approvaldate_319D': 36.67, 'byoccupationinc_...","{'coluna': 'case_id', 'status': 'inválida'}"
1,test_applprev_2,10,6,"[[case_id, bigint], [cacccardblochreas_147M, s...","{'count': {'case_id': '10', 'cacccardblochreas...","{'conts_type_509L': 40.0, 'credacc_cards_statu...","{'coluna': 'case_id', 'status': 'inválida'}"
2,test_credit_bureau_a_1,50,79,"[[case_id, bigint], [annualeffectiverate_199L,...","{'count': {'case_id': '50', 'annualeffectivera...","{'annualeffectiverate_199L': 74.0, 'annualeffe...","{'coluna': 'case_id', 'status': 'inválida'}"
3,test_credit_bureau_a_2,120,19,"[[case_id, bigint], [collater_typofvalofguaran...","{'count': {'case_id': '120', 'collater_typofva...","{'collater_valueofguarantee_1124L': 76.67, 'co...","{'coluna': 'case_id', 'status': 'inválida'}"
4,test_credit_bureau_b_1,10,45,"[[case_id, bigint], [amount_1115A, double], [c...","{'count': {'case_id': '10', 'amount_1115A': '4...","{'amount_1115A': 60.0, 'credlmt_1052A': 80.0, ...","{'coluna': 'case_id', 'status': 'inválida'}"
5,test_credit_bureau_b_2,10,6,"[[case_id, bigint], [num_group1, bigint], [num...","{'count': {'case_id': '10', 'num_group1': '10'...",{},"{'coluna': 'case_id', 'status': 'inválida'}"
6,test_static_0,30,168,"[[case_id, bigint], [actualdpdtolerance_344P, ...","{'count': {'case_id': '30', 'actualdpdtoleranc...","{'amtinstpaidbefduel24m_4187115A': 20.0, 'avgd...","{'coluna': 'case_id', 'status': 'válida'}"
7,test_base,10,4,"[[case_id, bigint], [date_decision, string], [...","{'count': {'case_id': '10', 'date_decision': '...",{},"{'coluna': 'case_id', 'status': 'válida'}"
8,test_debitcard_1,10,6,"[[case_id, bigint], [last180dayaveragebalance_...","{'count': {'case_id': '10', 'last180dayaverage...","{'last180dayaveragebalance_704A': 100.0, 'last...","{'coluna': 'case_id', 'status': 'inválida'}"
9,test_deposit_1,10,5,"[[case_id, bigint], [amount_416A, double], [co...","{'count': {'case_id': '10', 'amount_416A': '10...",{'contractenddate_991D': 60.0},"{'coluna': 'case_id', 'status': 'inválida'}"


## 📊 Resultado da descrição das tabelas
### 🧩 Estrutura da descrição

Baseado na estudo feito anteriormente, os datasets foram empilhados conforme os parâmetros estabelecidos no estudo gerando um panorâma geral das váriáveis de cada grupo característico de tabelas.

O arquivo resultante desse processo é um arquivo json. A escolha por esse formato foi mediante o grupo de informações que seria gerado. A estrutura do json se mostrou com melhor distribuição do que formatos tabulares.

A maioria das tabelas apresenta valores nulos em suas colunas o que aponta para avaliação da necessidade de tratamento ou descarte dessas variáveis.

A maioria das colunas não possuem chave primária (relacionamento 1:N). Mas possuem todas possuem a variável case_id. Essa variável será a chave primária após o tratamento necessário em cada tabela.

A tabela **train_base** será a tabela semente onde as outras tabelas, que se mostrarem agregadoras, serão agregadas para construção de uma tabela geral. 